In [90]:
import pandas as pd
import numpy as np
from IPython.display import HTML

data = pd.read_csv('../data/ERP_data.csv')
labels = list(data.columns.values)
del labels[0]
del labels[0]

targets = data['Phenotype']
del data['Subject']
del data['Phenotype']

In [91]:
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import cross_validation

folds = 10

imp = preprocessing.Imputer()
data = imp.fit_transform(data, targets)
data = preprocessing.scale(data)
data = feature_selection.SelectKBest(feature_selection.f_classif, k=21).fit_transform(data, targets)

In [94]:
from sklearn import linear_model

logreg_model = linear_model.LogisticRegression(C=0.95)

scores = []
stddev = []
for i in range(70):
    cross_val = cross_validation.KFold(n=198, n_folds=folds, shuffle=True)
    logreg_score = cross_validation.cross_val_score(logreg_model, data, targets, cv=cross_val)
    scores.append(np.mean(logreg_score))
    stddev.append(np.std(logreg_score))

print(np.mean(scores))
print(np.mean(stddev))

0.694266917293
0.0948497418604
